In [1]:
import os
import sys
import cv2

from pedect.converter.ConverterToImages import *
from pedect.config.BasicConfig import *
from pedect.service.Service import *
from pedect.tracker.OpenCVTracker import *
from typing import *
from pedect.predictor.PredictedBox import *
from pedect.utils.constants import *
from pedect.predictor.GroundTruthPredictor import *
from pedect.predictor.TrackerPredictor import *
from pedect.predictor.YoloPredictor import *

sys.path.append("./keras-yolo3/") 
sys.path.append("./re3-tensorflow/") 
print(sys.path)

Created ThreadPoolExecutor with 8 threads!


Using TensorFlow backend.


['E:\\licenta\\App', 'E:\\licenta\\App\\keras-retinanet', 'E:\\licenta\\App\\re3-tensorflow', 'E:\\licenta\\App\\mAP', 'E:\\licenta\\App\\keras-yolo3', 'C:\\Users\\sebin\\Anaconda3\\envs\\py36\\python36.zip', 'C:\\Users\\sebin\\Anaconda3\\envs\\py36\\DLLs', 'C:\\Users\\sebin\\Anaconda3\\envs\\py36\\lib', 'C:\\Users\\sebin\\Anaconda3\\envs\\py36', '', 'C:\\Users\\sebin\\AppData\\Roaming\\Python\\Python36\\site-packages', 'C:\\Users\\sebin\\Anaconda3\\envs\\py36\\lib\\site-packages', 'C:\\Users\\sebin\\Anaconda3\\envs\\py36\\lib\\site-packages\\IPython\\extensions', 'C:\\Users\\sebin\\.ipython', './keras-yolo3/', './re3-tensorflow/']


In [2]:
class MyConfig(BasicConfig):
    pass
config = getConfigFromTrainId(11)
config.trackerType = "csrt"
controller = Service(config)

In [3]:
vset = "set04"
vid = "V002"
video = ("caltech", vset, vid)
gtPredictor = GroundTruthPredictor(video[0], video[1], video[2])
yoloPredictor = YoloPredictor(gtPredictor, config)
trackerPredictor = TrackerPredictor(yoloPredictor, gtPredictor, controller.tracker, config)

Reading ground truth
The Reading might take a bit longer
Finished reading ground truth in  7.835131406784058


In [5]:
import copy
def addRectanglesToImage(image, predictedBBoxes: Sequence[PredictedBox], color: Sequence[int] = None):
    if color is None:
        color = [0, 255, 0]
    for predBox in predictedBBoxes:
        x1 = predBox.getX1()
        y1 = predBox.getY1()
        x2 = predBox.getX2() - x1
        y2 = predBox.getY2() - y1
        cv2.rectangle(image, (x1, y1, x2, y2), color)
RED = [0, 0, 255]
GREEN = [0, 255, 0]
BLUE = [255, 0, 0]
        
def recordVideo(videoName, predictors, videoHolder, noFrames = MAX_VIDEO_LENGTH, framesPerFrame = 1):
    cv2.namedWindow('Video', cv2.WINDOW_NORMAL)
    cv2.resizeWindow('Video', 640, 480)
    video = videoHolder.getVideo()
    finalVideoFrames = []
    for frameNr in range(min(videoHolder.getLength(), noFrames)):
        image = copy.deepcopy(video[frameNr][:])  # predictor.getFrame(frameNr)
        # print("Frame %d" % frameNr)
        l = []
        for predictor, color in predictors:
            predictedBBoxes = predictor.predictForFrame(frameNr)
            l.append(len(predictedBBoxes))
            
            addRectanglesToImage(image, predictedBBoxes, color)
        print(l)
        cv2.imshow('Video', image)
        cv2.waitKey(1)
        finalVideoFrames.append(image)
    cv2.destroyAllWindows()
    vid = make_video(finalVideoFrames, videoName)
    
def make_video(frames, videoName):
    import numpy as np
    import cv2

    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(videoName,fourcc, 20.0, (640,480))

    for frame in frames:
        # write the flipped frame
        out.write(frame)
        cv2.imshow('frame',frame)
        

# Release everything if job is finished
    out.release()
    cv2.destroyAllWindows()
trackerPredictor.tracker = OpenCVTracker("csrt")
controller.tracker = trackerPredictor.tracker
config = trackerPredictor.config
config.createThreshold = 0.0
config.removeThreshold = 0.4
config.surviveThreshold = 0.0
config.surviveMovePercent = 0.0
config.minScorePrediction = 0.0
print(trackerPredictor.config)

for i in tqdm(range(300)):
    len([str(x) for x in yoloPredictor.predictForFrame(i)])
recordVideo("recordedVideo.mp4", [(trackerPredictor, BLUE), (yoloPredictor, RED), (gtPredictor, GREEN)], gtPredictor, 300)
# recordVideo("recordedVideo.mp4", [(trackerPredictor, BLUE)], gtPredictor, 400)
trackerPredictor.activeObjects = {}

trainId = 11
modelName = trained_weights_final.h5
inputShape = (224, 320)
freezeNoEpochs = 50
noFreezeNoEpochs = 50
isTiny = True
validationSplit = 0.1
freezeBatchSize = 32
noFreezeBatchSize = 32
loadPretrained = True
checkpointPeriod = 5
initialLR = 0.0001
createThreshold = 0.0
removeThreshold = 0.4
surviveThreshold = 0.0
surviveMovePercent = 0.0
minScorePrediction = 0.0
maxNrOfObjectsPerFrame = 10
maxAge = 100
imageGenerationSavePeriod = 10
imageGenerationSavePath = ./../Data/images/predicted
imageGenerationSaveFileName = annotations.csv
batchSplit = (0.2, 0.1, 0.05, 0.65)
trackerType = csrt



100%|██████████████████████████████████████████████████████████████████████████████| 300/300 [00:00<00:00, 6947.28it/s]


[0, 0, 0]
[0, 0, 0]
[0, 0, 0]
[0, 0, 0]
[0, 0, 0]
[0, 0, 0]
[0, 0, 0]
[0, 0, 0]
[0, 0, 0]
[0, 0, 0]
[0, 0, 0]
[0, 0, 0]
[0, 0, 0]
[0, 0, 0]
[0, 0, 0]
[0, 0, 0]
[0, 0, 0]
[0, 0, 0]
[0, 0, 0]
[0, 0, 0]
[0, 0, 0]
[0, 0, 0]
[0, 0, 0]
[0, 0, 0]
[0, 0, 0]
[0, 0, 0]
[0, 0, 0]
[0, 0, 0]
[0, 0, 0]
[0, 0, 0]
[0, 0, 0]
[0, 0, 0]
[0, 0, 0]
[0, 0, 0]
[0, 0, 0]
[0, 0, 0]
[0, 0, 1]
[0, 0, 1]
[0, 0, 1]
[0, 0, 1]
[0, 0, 1]
[0, 0, 1]
[0, 0, 1]
[0, 0, 1]
[0, 0, 1]
[0, 0, 1]
[0, 0, 1]
[0, 0, 1]
[0, 0, 1]
[0, 0, 1]
[0, 0, 1]
[0, 0, 1]
[0, 0, 1]
[0, 0, 1]
[0, 0, 1]
[0, 0, 1]
[0, 0, 1]
[1, 1, 1]
[0, 0, 1]
[0, 0, 1]
[0, 0, 1]
[0, 0, 1]
[0, 0, 2]
[1, 1, 2]
[0, 0, 2]
[1, 1, 2]
[1, 1, 2]
[2, 1, 2]
[0, 0, 2]
[2, 2, 2]
[0, 0, 2]
[1, 1, 2]
[2, 2, 2]
[2, 1, 2]
[0, 0, 2]
[0, 0, 2]
[1, 1, 2]
[0, 0, 2]
[2, 2, 2]
[2, 1, 2]
[2, 1, 2]
[2, 2, 2]
[2, 1, 2]
[3, 1, 2]
[2, 1, 2]
[2, 1, 2]
[2, 1, 2]
[2, 1, 2]
[2, 1, 2]
[2, 1, 2]
[3, 2, 2]
[3, 2, 2]
[3, 1, 2]
[3, 1, 2]
[2, 1, 2]
[2, 1, 2]
[2, 1, 2]
[0, 0, 2]
[0, 0, 1]
[0, 0, 1]
